In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.utils.data import TensorDataset, DataLoader
from torchviz import make_dot
from torchsummary import summary

import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import re

from sklearn import svm
from sklearn.model_selection import train_test_split

import glob
import os
import time
import sys
from tqdm.notebook import tqdm

sys.path.append("/home/cygnus/fujimoto/Cygnus-X_Molecular_Cloud_Analysis/Binary_classification/training/models")
from Binary_classification import Binary_classification

In [2]:
class DataSet():
    def __init__(self, data, label):
        self.label = label
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index], self.label[index]

In [3]:
weight_path = "/home/cygnus/fujimoto/Cygnus-X_Molecular_Cloud_Analysis/Binary_classification/training/save_dir/binary_cheting_model.pth"
weight_para = torch.load(weight_path, map_location=torch.device('cpu'))

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 
print(f"使用デバイス: {device}")

使用デバイス: cuda:0


In [5]:
model = Binary_classification(latent=100, input_depth=30, input_height=100, input_width=100)
model.load_state_dict(weight_para)
model.to(device)
model.eval()

Binary_classification(
  (features): Sequential(
    (0): Conv3d(1, 16, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
    (1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv3d(16, 32, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
    (4): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (8): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv3d(32, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (11): ReLU(inplace=True)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1600, out_features=100, bias=True)
    (2): ReLU(inplace=True)
    (

In [6]:
# summary(model.to(device), (1, 30, 100, 100))

In [7]:
training_bubble = np.load("/home/cygnus/fujimoto/Cygnus-X_Molecular_Cloud_Analysis/Binary_classification/data/processed_data/split_area/training_data/training_bubble.npy")
training_data = np.load("/home/cygnus/fujimoto/Cygnus-X_Molecular_Cloud_Analysis/Binary_classification/data/processed_data/split_area/training_data/training_data.npy")

# バブルのラベルは1、非バブルのラベルは0
training_bubble_label = [1] * len(training_bubble)
training_data_label = [0] * len(training_data)

validation_bubble = np.load("/home/cygnus/fujimoto/Cygnus-X_Molecular_Cloud_Analysis/Binary_classification/data/processed_data/split_area/validation_data/validation_cheating_bubble.npy")
validation_data = np.load("/home/cygnus/fujimoto/Cygnus-X_Molecular_Cloud_Analysis/Binary_classification/data/processed_data/split_area/validation_data/validation_cheating_data.npy")

# バブルのラベルは1、非バブルのラベルは0
validation_bubble_label = [1] * len(validation_bubble)
validation_data_label = [0] * len(validation_data)

test_bubble = np.load("/home/cygnus/fujimoto/Cygnus-X_Molecular_Cloud_Analysis/Binary_classification/data/processed_data/split_area/test_data/test_bubble.npy")
test_data = np.load("/home/cygnus/fujimoto/Cygnus-X_Molecular_Cloud_Analysis/Binary_classification/data/processed_data/split_area/test_data/test_data.npy")

# バブルのラベルは1、非バブルのラベルは0
test_bubble_label = [1] * len(test_bubble)
test_data_label = [0] * len(test_data)

In [7]:
# bubble_data = np.load("/home/cygnus/fujimoto/Cygnus-X_Molecular_Cloud_Analysis/Binary_classification/data/processed_data/slide_bubble.npy")
# removal_data = np.load("/home/cygnus/fujimoto/Cygnus-X_Molecular_Cloud_Analysis/Binary_classification/data/processed_data/all_data_after_bubble_removal.npy")

# # バブルのラベルは1、非バブルのラベルは0
# bubble_label = [1] * len(bubble_data)
# removal_label = [0] * len(removal_data)

In [8]:
data = np.concatenate((training_data, validation_data))
label = np.concatenate((training_data_label, validation_data_label))

bubble_data = np.concatenate((training_bubble, validation_bubble))
bubble_label = np.concatenate((training_bubble_label, validation_bubble_label))

In [11]:
dataset = DataSet(data, label)
dataloader = DataLoader(dataset, batch_size=16, shuffle=False)

bubble_dataset = DataSet(bubble_data, bubble_label)
bubble_dataloader = DataLoader(bubble_dataset, batch_size=16, shuffle=False)

test_dataset = DataSet(test_data, test_data_label)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

test_bubble_dataset = DataSet(test_bubble, test_bubble_label)
test_bubble_dataloader = DataLoader(test_bubble_dataset, batch_size=16, shuffle=False)

In [ ]:
prediction_list = []
latent_list = []
model = model.to("cpu")
for images, labels in tqdm(dataloader):
    images = images.view(-1, 1, 30, 100, 100).to("cpu")  # バッチサイズを維持したままチャンネル数を1に設定
    labels = labels.to("cpu").float()

    with torch.set_grad_enabled(True):
        # モデルの出力を計算する
        images = images.float()
        prediction, latent_vector = model(images.clone())
        prediction_list.append(prediction.to("cpu").detach().numpy())
        latent_list.append(latent_vector.to("cpu").detach().numpy())

prediction_list = np.concatenate(prediction_list)
latent_list = np.concatenate(latent_list)

print(len(prediction_list))
print(len(latent_list))

np.save("/home/cygnus/fujimoto/Cygnus-X_Molecular_Cloud_Analysis/Binary_classification/analysis/check_latent/latent_list/cheating_model/data_latent.npy", latent_list)

  0%|          | 0/276 [00:00<?, ?it/s]

In [11]:
prediction_list = []
latent_list = []
model = model.to("cpu")
for images, labels in tqdm(bubble_dataloader):
    images = images.view(-1, 1, 30, 100, 100).to("cpu")  # バッチサイズを維持したままチャンネル数を1に設定
    labels = labels.to("cpu").float()

    with torch.set_grad_enabled(True):
        # モデルの出力を計算する
        images = images.float()
        prediction, latent_vector = model(images.clone())
        prediction_list.append(prediction.to("cpu").detach().numpy())
        latent_list.append(latent_vector.to("cpu").detach().numpy())

prediction_list = np.concatenate(prediction_list)
latent_list = np.concatenate(latent_list)

print(len(prediction_list))
print(len(latent_list))

np.save("/home/cygnus/fujimoto/Cygnus-X_Molecular_Cloud_Analysis/Binary_classification/analysis/check_latent/latent_list/cheating_model/bubble_data_latent.npy", latent_list)

  0%|          | 0/337 [00:00<?, ?it/s]

5380
5380


In [ ]:
prediction_list = []
latent_list = []
model = model.to("cpu")
for images, labels in tqdm(test_dataloader):
    images = images.view(-1, 1, 30, 100, 100).to("cpu")  # バッチサイズを維持したままチャンネル数を1に設定
    labels = labels.to("cpu").float()

    with torch.set_grad_enabled(True):
        # モデルの出力を計算する
        images = images.float()
        prediction, latent_vector = model(images.clone())
        prediction_list.append(prediction.to("cpu").detach().numpy())
        latent_list.append(latent_vector.to("cpu").detach().numpy())

prediction_list = np.concatenate(prediction_list)
latent_list = np.concatenate(latent_list)

print(len(prediction_list))
print(len(latent_list))

np.save("/home/cygnus/fujimoto/Cygnus-X_Molecular_Cloud_Analysis/Binary_classification/analysis/check_latent/latent_list/cheating_model/test_data_latent.npy", latent_list)

In [ ]:
prediction_list = []
latent_list = []
model = model.to("cpu")
for images, labels in tqdm(test_bubble_dataloader):
    images = images.view(-1, 1, 30, 100, 100).to("cpu")  # バッチサイズを維持したままチャンネル数を1に設定
    labels = labels.to("cpu").float()

    with torch.set_grad_enabled(True):
        # モデルの出力を計算する
        images = images.float()
        prediction, latent_vector = model(images.clone())
        prediction_list.append(prediction.to("cpu").detach().numpy())
        latent_list.append(latent_vector.to("cpu").detach().numpy())

prediction_list = np.concatenate(prediction_list)
latent_list = np.concatenate(latent_list)

print(len(prediction_list))
print(len(latent_list))

np.save("/home/cygnus/fujimoto/Cygnus-X_Molecular_Cloud_Analysis/Binary_classification/analysis/check_latent/latent_list/cheating_model/test_bubble_data_latent.npy", latent_list)